最近[equinox](https://docs.kidger.site/equinox/)という[jax](https://jax.readthedocs.io/en/latest/)ベースの深層学習モデルを定義するライブラリを使ってみたのですが、これが中々いいと思ったので紹介ついでに強化学習してみます。他のjaxベースのライブラリにはDeepmindの[haiku]()やGoogle Researchの[flax]()があります。この2つのライブラリは実際のところあまり変わりはありません。というのも、jaxには試験的に書かれた[stax](https://jax.readthedocs.io/en/latest/jax.example_libraries.stax.html)という深層学習ライブラリのリファレンス実装があり、haikuもflaxもstaxをベースにオブジェクト志向的な`Module`を採用したものだからです。あるいは、haikuやflaxは「staxをPyTorchっぽくしたもの」と言ってもいいかもしれません。equinoxのドキュメントにある[Compatibility with init-apply libraries](https://docs.kidger.site/equinox/examples/init_apply/)というページでは、これらのライブラリのやり方を「init-applyアプローチ」と呼んで軽く説明しています。これについてざっと見てみましょう。

# Jaxにできることとできないこと

そもそも、jaxというのは何をしてくれるライブラリなのでしょうか。ホームページのトップにはこう書かれています。

> JAX is Autograd and XLA, brought together for high-performance numerical computing.

[XLA](https://www.tensorflow.org/xla)というのは、Tensorflowのバックエンドとして開発された深層学習用の中間言語で、CPU/GPU/TPU用に数値計算コードを最適化してくるものです。深層学習に求められる並列化の性質から、特にSIMD演算/ベクトル並列化に特化しています。Jaxは、`jax.numpy`というNumPyに似せたライブラリをXLAのフロントエンドとして提供することで、「NumPyコードをベクトル並列化された高速なGPU用コードに実行時コンパイルすること」を可能にしています。では、Autogradというのはなんでしょうか？これは、jaxの開発者が以前に開発していたライブラリの名前でもありますが、自動微分全般のことを指すと考えていいでしょう。自分で勾配逆伝播のコードを書かなくても、jaxでは損失関数の各パラメタにおいての偏微分を勝手に計算してくれます。試しに、$f(x, y) = x^2 + y$の偏微分を計算してみましょう。

In [1]:
import jax
import jax.numpy as jnp

def f(x: jax.Array, y: jax.Array) -> jax.Array:
    return jnp.sum(x ** 2 + y)

x = jnp.array([3, 4, 5, 6, 7], dtype=jnp.float32)
y = jnp.array([5, 2, 5, 7, 2], dtype=jnp.float32)
jax.grad(f, argnums=(0, 1))(x, y)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


(Array([ 6.,  8., 10., 12., 14.], dtype=float32),
 Array([1., 1., 1., 1., 1.], dtype=float32))

$\frac{\partial}{\partial x}f = 2x, \frac{\partial}{\partial y}f = 1$なので、正しく計算されているようですね。こんな感じで、jaxは自動的に偏微分を計算してくれるので、これをそのまま勾配降下法に使ってモデルを学習させることができます。深層ニューラルネットワークを学習させる際も、この`grad`を使えば全部のパラメタについて偏微分を効率的に計算してくれるので、それを使って学習できます。ついでに、この勾配をとる計算やモデルを更新する計算を`jax.jit`につっこめば、高速に計算してくれます。
ここで問題になるのは、深層ニューラルネットワークの場合パラメタが多すぎるので、こんな風にいちいち全てのパラメタについて変数を割り当ててプログラムを書いていたら大変すぎる、ということです。大変すぎる以外に何か問題はあるのかというと、特にないと思います。コードの再利用性くらいでしょうか。しかし、まあ面倒なものは面倒ですから、**パラメタを管理してくれる仕組み**がほしいなあ、と思うわけですね。

# init-applyアプローチによるパラメタ管理

stax, haiku, flaxでは、この「パラメタ管理の問題」を、「init-applyアプローチ」により解決しています。このアプローチは以下のようにまとめられます。
1. モデルはパラメータを持たない
2. モデルは`init`と`apply`の2つの関数を持つ
  - `init`は、入力例を受け取ってパラメタを初期化し、最初のパラメータを返す
  - `apply`は、入力とパラメタを受け取り、モデルの計算結果を返す
  
なので、flaxやhaikuのAPIは以下のような感じになります。flaxでは`__call__`を使うがhaikuはPyTorchと同じ`forward`を使う、`flax`のModuleは [`dataclasses.dataclass`](https://docs.python.org/ja/3/library/dataclasses.html#dataclasses.dataclass)デコレータにより定義されたクラスと同じような性質を持つなどの違いがありますが、まあそれくらいで、大して違いはないです。以下僕がflaxとhaikuの間をとって書いた適当な疑似コードです。ニューラルネットワークを表すクラスとして、PyTorch風に「Module」という名前を使っています。
```python
class Linear(Module):
    def __call__(self, x):
        batch_size = x.shape[0]
        w = self.parameter(output, shape=(batch_size, self.output_size) init=self.w_init)
        b = self.parameter(output, shape=(1, self.output_size) init=self.w_init)
        return w @ x.T + b

model = Linear(output_size=10)
params = model.init(jnp.zeros(1, 100))
result = model.apply(params, jnp.zeros(10, 100))
```
こんな感じですかね。なお、上疑似コード中の`self.parameter`というメソッドは`flax`や`haiku`にある「パラメータをクラスに登録する機能」のことです。この機能により、各パラメタを値としてもつ`dict`を`init`により返すことができます。haikuの場合は、`params`は以下のような`dict`になっています。
```python
{
    "Linear/~/w": [...],
    "Linear/~/b": [...],
}
```

`stax`はただのreference implementationなのでこのような機能がなく、かわりに、数のレイヤーを組み合わせるコンビネーターを提供しています。ではこのアプローチにはどのようなメリット、デメリットがあるでしょうか。

**メリット**
- Moduleを初期化する際に入力されるArrayのshapeを指定しなくていい
  - パラメタは`init`が呼ばれた際に初期化される
- 関数とデータを分離できる
  - Moduleは変更可能な変数を持たず、パラメタと完全に別に扱われる
- `init`、`apply`に`jit`や`vmap`などの関数デコレータを適用するコードが自然に書ける
  
**デメリット**
- Moduleは冗長
  - Moduleは「出力の次元」などの「モデルに関する設定」を持っているだけ
- パラメタの要素に直接アクセスするのが面倒
  - 例えばhaikuなら`params["Linear/~/w"]`のようにしてパラメタの各要素にアクセスできるが、複雑なクラスだと`dict`の鍵の名前が長くなりわかりにくい
- あまりオブジェクト指向的ではない
- (haiku/flaxに特有) Module内でのパラメタの呼び出しを`jax.grad`が使えるような関数に変換する必要がある
  - 例えば`haiku.transform`は、`haiku.get_parameter`によるパラメタ呼び出しを含む関数を、パラメタを引数としてとる関数に変換する

こんなところでしょうか。

# equinoxの特徴

equinoxの特徴は、init-applyアプローチと異なり、「よりオブジェクト指向的な」（または、PyTorchに近い）インターフェースを志向している点にあります。
ドキュメントのトップページにある[Quick example](https://docs.kidger.site/equinox/#quick-example)を見てみましょう。

In [2]:
import equinox as eqx
import jax

class Linear(eqx.Module):
    weight: jax.Array
    bias: jax.Array

    def __init__(self, in_size, out_size, key):
        wkey, bkey = jax.random.split(key)
        self.weight = jax.random.normal(wkey, (out_size, in_size))
        self.bias = jax.random.normal(bkey, (out_size,))

    def __call__(self, x):
        return self.weight @ x + self.bias
    
@jax.jit
@jax.grad
def loss_fn(model, x, y):
    pred_y = jax.vmap(model)(x)
    return jax.numpy.mean((y - pred_y) ** 2)

batch_size, in_size, out_size = 32, 2, 3
model = Linear(in_size, out_size, key=jax.random.PRNGKey(0))
x = jax.numpy.zeros((batch_size, in_size))
y = jax.numpy.zeros((batch_size, out_size))
grads = loss_fn(model, x, y)
print("Model weight?", model.weight)
print("Grad?", grads)

Model weight? [[0.59902626 0.2172144 ]
 [0.660603   0.03266738]
 [1.2164948  1.1940813 ]]
Grad? Linear(weight=f32[3,2], bias=f32[3])


ここで、equinoxの最大の特徴は
```python
class Linear(eqx.Module):
    weight: jax.Array
    bias: jax.Array
```
というコードに表れていますが、「Moduleがパラメタを直接持つ」という点です。init-applyアプローチと比べると、これはどのような利点・欠点があるでしょうか？

**メリット**
- わかりやすい
- デバッグしやすい
  - flax/haikuは一度`transform`しないとデバッグできない
- パラメタを直接操作するのが簡単

**デメリット**
- モデルを初期化する時に、入力する特徴量の次元が必要
- `grad`を使う際に、パラメタとその他の変数を分離する必要がある

ここで、最後の「`grad`を使う際に、パラメタとその他の変数を分離する必要がある」というのは、どういう意味でしょうか？例えば、self-attentionを計算する以下のようなModuleを考えます。

In [3]:
class SelfAttention(eqx.Module):
    q: eqx.nn.Linear
    k: eqx.nn.Linear
    v: eqx.nn.Linear
    sqrt_d_attn: float

    def __init__(self, d_in: int, d_attn: int, d_out: int, key: jax.Array) -> None:
        q_key, k_key, v_key = jax.random.split(key, 3)
        self.q = eqx.nn.Linear(d_in, d_attn, key=q_key)
        self.k = eqx.nn.Linear(d_in, d_attn, key=k_key)
        self.v = eqx.nn.Linear(d_in, d_attn, key=k_key)
        self.sqrt_d_attn = float(jnp.sqrt(d_attn))

    def __call__(self, e: jax.Array) -> jax.Array:
        q = jax.vmap(self.q)(e)
        k = jax.vmap(self.k)(e)
        alpha = jax.nn.softmax(q.T @ k / self.sqrt_d_attn, axis=-1)
        return jax.vmap(self.v)(e) @ alpha.T

`eqx.Module`は内部で`dataclasses.dataclass`を使うので、`__init__`等の初期化メソッドはあらかじめ用意されていますが、これをオーバーライドしてパラメタの初期化に使います。
$\sqrt{d_\mathrm{attn}}$は定数なので、これもメンバ変数にしてしまいましょう。勾配を計算してみます。

In [4]:
model = SelfAttention(4, 8, 4, jax.random.PRNGKey(10))
jax.grad(lambda model, x: jnp.mean(model(x)))(model, jnp.ones((3, 4)))

SelfAttention(
  q=Linear(
    weight=f32[8,4],
    bias=f32[8],
    in_features=4,
    out_features=8,
    use_bias=True
  ),
  k=Linear(
    weight=f32[8,4],
    bias=f32[8],
    in_features=4,
    out_features=8,
    use_bias=True
  ),
  v=Linear(
    weight=f32[8,4],
    bias=f32[8],
    in_features=4,
    out_features=8,
    use_bias=True
  ),
  sqrt_d_attn=f32[]
)

ここで困ったことに、`sqrt_d_attn`での偏微分も計算されてしまいました。`eqx.Module`そのものがパラメタを持つことによって、定数であるようなメンバ変数に対しても偏微分が計算されてしまいます。この問題を、equinoxでは、[`eqx.partition`](https://docs.kidger.site/equinox/api/filtering/partition-combine/#equinox.partition)と[`eqx.is_inexact_array`](https://docs.kidger.site/equinox/api/filtering/partition-combine/#equinox.is_inexact_array)を使って、「32bit浮動小数点の`jax.Array`または`numpy.ndarray`」と「その他のメンバ変数」を分離することにより解決しています。やってみましょう。

In [5]:
eqx.partition(model, eqx.is_inexact_array)

(SelfAttention(
   q=Linear(
     weight=f32[8,4],
     bias=f32[8],
     in_features=4,
     out_features=8,
     use_bias=True
   ),
   k=Linear(
     weight=f32[8,4],
     bias=f32[8],
     in_features=4,
     out_features=8,
     use_bias=True
   ),
   v=Linear(
     weight=f32[8,4],
     bias=f32[8],
     in_features=4,
     out_features=8,
     use_bias=True
   ),
   sqrt_d_attn=None
 ),
 SelfAttention(
   q=Linear(weight=None, bias=None, in_features=4, out_features=8, use_bias=True),
   k=Linear(weight=None, bias=None, in_features=4, out_features=8, use_bias=True),
   v=Linear(weight=None, bias=None, in_features=4, out_features=8, use_bias=True),
   sqrt_d_attn=2.8284270763397217
 ))

`sqrt_d_attn=None`のものと、全てのパラメタがNoneで`sqrt_d_attn=2.8284270763397217`のものに分離されています。なので、定数を持つModuleに対して勾配を計算したい場合は
1. Moduleをパラメタとそれ以外に分割
2. 勾配を求めたい関数`f(module, ...)`をラップする関数`g(params, others, ..)`みたいなものを作る
3. `jax.grad(g)(params, others, ...)`で勾配を計算
という流れになります。面倒ですね。
長々説明したのですが、これを全部やってくれるのが、`equinox.filter_value_and_grad`です。基本的にもうこれを使えばいいです。やってみましょう。

In [6]:
eqx.filter_value_and_grad(lambda model, x: jnp.mean(model(x)))(model, jnp.ones((3, 4)))

(Array(-0.06870805, dtype=float32),
 SelfAttention(
   q=Linear(
     weight=f32[8,4],
     bias=f32[8],
     in_features=4,
     out_features=8,
     use_bias=True
   ),
   k=Linear(
     weight=f32[8,4],
     bias=f32[8],
     in_features=4,
     out_features=8,
     use_bias=True
   ),
   v=Linear(
     weight=f32[8,4],
     bias=f32[8],
     in_features=4,
     out_features=8,
     use_bias=True
   ),
   sqrt_d_attn=None
 ))

計算結果と勾配を返してくれました。定数の`sqrt_d_attn`はきっちり`None`でマスクされています。なので、Moduleで定数を持ちたかったらとりあえず`jax.Array`か`ndarray`以外の型にしておけばいいです。じゃあ32bit浮動小数点型の`jax.Array`を定数として持ちたかったらどうすればいいんだというと、`filter_value_and_grad`を直接使えずめちゃくちゃ面倒になるので、避けたほうが良さげです。ただPython組み込みの`float`や`bool`もjitコンパイルしてしまえばただの定数になるので、パフォーマンス的には気にする必要はないです。

# 強化学習してみる

## 環境

一通り`equinox`の特徴をおさらいしたところで、これを使って強化学習してみます。せっかくjaxを使っているの~~とgymのAPIが変わりまくった上にgymnasiumに変わって全然ついていけないの~~で、jax製の環境を使ってみましょう。ここでは[jumanji](https://instadeepai.github.io/jumanji/)というライブラリの[Maze](https://instadeepai.github.io/jumanji/environments/maze/?h=maze)を使ってみます。

In [85]:
import jumanji
from jumanji.wrappers import AutoResetWrapper
from IPython.display import HTML

env = jumanji.make("Maze-v0")
env = AutoResetWrapper(env)
n_actions = env.action_spec().num_values
key, *keys = jax.random.split(jax.random.PRNGKey(43), 11)
state, timestep = env.reset(key1)
states = [state]
for key in keys:
    action = jax.random.randint(key=key, minval=0, maxval=n_actions, shape=())
    state, timestep = env.step(state, action)
    states.append(state)
anim = env.animate(states)
HTML(anim.to_html5_video().replace('="1000"', '="640"'))

<IPython.core.display.Javascript object>

使いやすそうですね。実際に学習するときには`vmap`や`jit`と組み合わせて使えるようです。

正直このVisualizerのAPIはどうなんだろう？という気もしますが...きちんとFour Roomsを表示してくれました。

## PPOを実装してみる
ということで、この環境を学習してみましょう。ここでは定番かつ学習が高速な[PPO](https://arxiv.org/abs/1707.06347)を実装してみます。
まずは、ネットワークを定義してみましょう。

In [117]:
from typing import NamedTuple

from jax.nn.initializers import orthogonal


class PPONetOutput(NamedTuple):
    policy_logits: jax.Array
    value: jax.Array


class SoftmaxPPONet(eqx.Module):
    torso: list
    value_head: eqx.nn.Linear
    policy_head: eqx.nn.Linear

    def __init__(self, key: jax.Array, n_actions: int) -> None:
        key1, key2, key3, key4, key5 = jax.random.split(key, 5)
        # Common layers
        self.torso = [
            eqx.nn.Conv2d(3, 3, kernel_size=4, key=key1),
            jax.nn.relu,
            jnp.ravel,
            eqx.nn.Linear(3 * 7 * 7, 64, key=key2),
            jax.nn.relu,
        ]
        self.value_head = eqx.nn.Linear(64, 1, key=key3)
        policy_head = eqx.nn.Linear(64, n_actions, key=key4)
        # Use small value for policy initialization
        self.policy_head = eqx.tree_at(
            lambda linear: linear.weight,
            policy_head,
            orthogonal(scale=0.01)(key5, policy_head.weight.shape),
        )

    def __call__(self, x: jax.Array) -> jax.Array:
        for layer in self.torso:
            x = layer(x)
        value = self.value_head(x)
        policy_logits = self.policy_head(x)
        return PPONetOutput(policy_logits=policy_logits, value=value)

In [168]:
@chex.dataclass
class Rollout:
    """Rollout buffer that stores the entire history of one rollout"""
    
    observations: jax.Array
    actions: jax.Array
    rewards: jax.Array
    terminations: jax.Array
    values: jax.Array
    policy_logits: jax.Array

In [169]:
from jumanji.environments.routing.maze.types import Observation, State

def obs_to_image(obs: Observation) -> jax.Array:
    walls = obs.walls.astype(jnp.float32)
    agent = jnp.zeros_like(walls).at[obs.agent_position].set(1.0)
    target = jnp.zeros_like(walls).at[obs.target_position].set(1.0)
    return jnp.stack([walls, agent, target])

In [170]:
def exec_rollout(
    initial_state: State,
    initial_obs: jax.Array,
    network: SoftmaxPPONet,
    prng_key: jax.Array,
    env: jumanji.Environment,
    n_rollout_steps: int,
) -> Rollout:
    n_actions = env.action_spec()
    keys = jax.random.split(prng_key, n_rollout_steps)

    def step_rollout(
        carried: tuple[State, jax.Array],
        key: jax.Array,
    ) -> tuple[tuple[State, jax.Array], Rollout]:
        state, obs = carried
        values, policy_logits = jax.vmap(network)(obs)
        actions = jax.random.categorical(key, policy_logits)
        state, timestep = jax.vmap(env.step)(state, actions)
        obs = jax.vmap(obs_to_image)(timestep.observation)
        rollout = Rollout(
            observations=obs,
            actions=actions,
            rewards=timestep.reward,
            terminations=timestep.discount,
            values=values,
            policy_logits=policy_logits
        )
        return (state, obs), rollout

    return jax.lax.scan(step_rollout, (initial_state, initial_obs), keys)

In [171]:
state, timestep = jax.vmap(env.reset)(jax.random.split(jax.random.PRNGKey(43), 10))

In [172]:
jax.vmap(obs_to_image)(timestep.observation).shape

(10, 3, 10, 10)

In [ ]:
exec_rollout(
    state,
    jax.vmap(obs_to_image)(timestep.observation),
    pponet,
    jax.random.PRNGKey(43),
    env,
    1024,
)

In [123]:
image = obs_to_image(timestep.observation)

In [115]:
pponet = SoftmaxPPONet(key=jax.random.PRNGKey(43), n_actions=n_actions)

In [116]:
pponet(image)

PPONetOutput(policy_logits=Array([-0.10854915,  0.12029472,  0.03809793, -0.01860824], dtype=float32), value=Array([-0.02617188], dtype=float32))

In [132]:
type(state)

jumanji.environments.routing.maze.types.State

In [136]:
timestep

TimeStep(step_type=Array(1, dtype=int8), reward=Array(0., dtype=float32), discount=Array(1., dtype=float32), observation=Observation(agent_position=Position(row=Array(2, dtype=int32), col=Array(0, dtype=int32)), target_position=Position(row=Array(2, dtype=int32), col=Array(8, dtype=int32)), walls=Array([[False,  True, False, False, False,  True, False,  True, False,
        False],
       [False,  True,  True,  True, False,  True, False,  True, False,
         True],
       [False, False, False,  True, False,  True, False,  True, False,
        False],
       [False,  True, False,  True, False,  True, False,  True, False,
         True],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False,  True,  True,  True,  True,  True,  True,  True, False,
         True],
       [False, False, False, False, False, False, False,  True, False,
        False],
       [False,  True, False,  True,  True,  True, False,  True, False,
         True],
      

In [77]:
key

Array([ 499813607, 2392722011], dtype=uint32)

In [127]:
jumanji.Environment

jumanji.env.Environment

In [128]:
env

AutoResetWrapper(Maze environment:
 - num_rows: 10
 - num_cols: 10
 - time_limit: 100
 - generator: <jumanji.environments.routing.maze.generator.RandomGenerator object at 0x7fbb97d7c450>)

In [131]:
env.observation_spec().

ObservationSpec(
	agent_position=PositionSpec(
	row=BoundedArray(shape=(), dtype=dtype('int32'), name='row_coordinate', minimum=Array(0, dtype=int32), maximum=Array(9, dtype=int32)),
	col=BoundedArray(shape=(), dtype=dtype('int32'), name='col_coordinate', minimum=Array(0, dtype=int32), maximum=Array(9, dtype=int32)),
),
	target_position=PositionSpec(
	row=BoundedArray(shape=(), dtype=dtype('int32'), name='row_coordinate', minimum=Array(0, dtype=int32), maximum=Array(9, dtype=int32)),
	col=BoundedArray(shape=(), dtype=dtype('int32'), name='col_coordinate', minimum=Array(0, dtype=int32), maximum=Array(9, dtype=int32)),
),
	walls=BoundedArray(shape=(10, 10), dtype=dtype('bool'), name='walls', minimum=Array(False, dtype=bool), maximum=Array(True, dtype=bool)),
	step_count=Array(shape=(), dtype=dtype('int32'), name='step_count'),
	action_mask=BoundedArray(shape=(4,), dtype=dtype('bool'), name='action_mask', minimum=Array(False, dtype=bool), maximum=Array(True, dtype=bool)),
)

In [49]:
def obs_to_image(obs: jumanji.environments.routing.maze.types.Observation) -> jax.Array:
    walls = obs.walls.astype(jnp.float32)
    agent_index
    agent = jnp.zeros_like(walls).at[obs.agent_position].set(1.0)
    target = jnp.zeros_like(walls).at[obs.target_position].set(1.0)
    return jnp.stack([walls, agent, target])

In [42]:
walls = timestep.observation.walls.astype(jnp.float32)

In [ ]:
pos = jnp.array([timestep.observation_position.row, timestep.observation.agent_position.col]) 

In [ ]:
jnp.zeros_like()

In [119]:
timestep.discount

Array(1., dtype=float32)

Observation(agent_position=Position(row=Array(2, dtype=int32), col=Array(0, dtype=int32)), target_position=Position(row=Array(2, dtype=int32), col=Array(8, dtype=int32)), walls=Array([[False,  True, False, False, False,  True, False,  True, False,
        False],
       [False,  True,  True,  True, False,  True, False,  True, False,
         True],
       [False, False, False,  True, False,  True, False,  True, False,
        False],
       [False,  True, False,  True, False,  True, False,  True, False,
         True],
       [False,  True, False, False, False, False, False, False, False,
        False],
       [False,  True,  True,  True,  True,  True,  True,  True, False,
         True],
       [False, False, False, False, False, False, False,  True, False,
        False],
       [False,  True, False,  True,  True,  True, False,  True, False,
         True],
       [False,  True, False,  True, False, False, False,  True, False,
        False],
       [False,  True,  True,  True, Fals